In [1]:
# Import starting packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

# import datasets from the unzipped CSVs
actual_demand_file_path = '../data/totaldemand_nsw.csv'
temperature_file_path = '../data/temperature_nsw.csv'
actual_demand = pd.read_csv(actual_demand_file_path, parse_dates=['DATETIME'])
temperature = pd.read_csv(temperature_file_path, parse_dates=['DATETIME'])

## Uncomment this for 2022 and 2023 data combining
# path = '../data/nswdemand2022onwards'
# all_files = glob.glob(os.path.join(path, "*.csv"))

# total_demand_2022onwards = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
# total_demand_2022onwards.head()


# Quick removal of certain columns and removal of incorrect -9999 values from the temperature data
actual_demand.drop(columns='REGIONID', inplace=True)
temperature.loc[temperature['TEMPERATURE'] <= -5, 'TEMPERATURE'] = np.nan
temperature.drop(columns='LOCATION', inplace=True)

# Combine temperature and demand
df = pd.merge(actual_demand,temperature,left_on=['DATETIME'], right_on=['DATETIME'], how = 'left')

# Interpolate missing temperature values
df['temperature'] = df['TEMPERATURE'].interpolate(method='linear')

# Remove extra temperature column and edit column names
df.drop(columns='TEMPERATURE', inplace= True)
df.rename(columns={'DATETIME': 'datetime', 'TOTALDEMAND':'demand'}, inplace=True)


# Create date and time variables using the datetime column
df['Is weekday'] = np.where(df['datetime'].dt.dayofweek <= 4, 1, 0)
df['Is weekend'] = np.where(df['datetime'].dt.dayofweek > 4, 1, 0)

for i in range(0,24):
    df[f'hour_{i}'] = (df['datetime'].dt.hour == i).astype(int)

for i in range(1,13):
    df[f'month_{i}'] = (df['datetime'].dt.month == i).astype(int)


## CREATE Lag and summary features for use in models
# create lag of demand and summaries of the demand lag to use as features in the model
df['demand_5_min_lag'] = df['demand'].shift(1)
df['demand_30_min_lag'] = df['demand'].shift(6)
df['demand_1_hr_lag'] = df['demand'].shift(12)
df['demand_24_hr_lag'] = df['demand'].shift(12*24)
df['demand_30_min_mean'] = df['demand'].rolling(window = 6).mean()
df['demand_1_hr_mean'] = df['demand'].rolling(window = 12).mean()
df['demand_24_hr_mean'] = df['demand'].rolling(window = 12*24).mean()
df['demand_30_min_std'] = df['demand'].rolling(window = 6).std()
df['demand_1_hr_std'] = df['demand'].rolling(window = 12).std()
df['demand_24_hr_std'] = df['demand'].rolling(window = 12*24).std()
df['demand_30_min_max'] = df['demand'].rolling(window = 6).max()
df['demand_1_hr_max'] = df['demand'].rolling(window = 12).max()
df['demand_24_hr_max'] = df['demand'].rolling(window = 12*24).max()
df['demand_30_min_min'] = df['demand'].rolling(window = 6).min()
df['demand_1_hr_min'] = df['demand'].rolling(window = 12).min()
df['demand_24_hr_min'] = df['demand'].rolling(window = 12*24).min()
# create lag of temperature and summaries of the temperature lag to use as features in the model
df['temperature_5_min_lag'] = df['temperature'].shift(1)
df['temperature_30_min_lag'] = df['temperature'].shift(6)
df['temperature_1_hr_lag'] = df['temperature'].shift(12)
df['temperature_24_hr_lag'] = df['temperature'].shift(12*24)
df['temperature_30_min_mean'] = df['temperature'].rolling(window = 6).mean()
df['temperature_1_hr_mean'] = df['temperature'].rolling(window = 12).mean()
df['temperature_24_hr_mean'] = df['temperature'].rolling(window = 12*24).mean()
df['temperature_30_min_std'] = df['temperature'].rolling(window = 6).std()
df['temperature_1_hr_std'] = df['temperature'].rolling(window = 12).std()
df['temperature_24_hr_std'] = df['temperature'].rolling(window = 12*24).std()
df['temperature_30_min_max'] = df['temperature'].rolling(window = 6).max()
df['temperature_1_hr_max'] = df['temperature'].rolling(window = 12).max()
df['temperature_24_hr_max'] = df['temperature'].rolling(window = 12*24).max()
df['temperature_30_min_min'] = df['temperature'].rolling(window = 6).min()
df['temperature_1_hr_min'] = df['temperature'].rolling(window = 12).min()
df['temperature_24_hr_min'] = df['temperature'].rolling(window = 12*24).min()
df.head(20)

# Add public holidays to the dataframe
public_holiday_path = '../data/nsw_public_holidays.csv'
public_holidays = pd.read_csv(public_holiday_path, parse_dates=['Date'])
df['date'] = pd.to_datetime(df['datetime'].dt.date)
df = pd.merge(df,public_holidays,left_on=['date'], right_on=['Date'], how = 'left')
df['regular day'] = np.where(df['Holiday Name'].isnull(), 1, 0)
df['public holiday'] = np.where(df['Holiday Name'].isnull(), 0, 1)
df.drop(columns=['date', 'Date', 'Holiday Name'], inplace = True)

# Write to final_data.csv

df.to_csv('../data/final_data.csv',index=False, mode = 'w')

df.head()


,datetime,demand,temperature,Is weekday,Is weekend,hour_0,hour_1,hour_2,hour_3,hour_4,...,temperature_1_hr_std,temperature_24_hr_std,temperature_30_min_max,temperature_1_hr_max,temperature_24_hr_max,temperature_30_min_min,temperature_1_hr_min,temperature_24_hr_min,regular day,public holiday
0,2010-01-01 00:00:00,7997.41,23.100000,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,2010-01-01 00:05:00,7946.51,23.066667,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,2010-01-01 00:10:00,7921.55,23.033333,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,2010-01-01 00:15:00,7837.84,23.000000,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,2010-01-01 00:20:00,7781.11,22.966667,1,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
